<a href="https://colab.research.google.com/github/mamta691/review_scrap_aws/blob/main/Copy_of_sparknlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=18c74267735cf1c18e342f9f5b4c0739a4ebc47021bb791010bbdd180fd323b0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install spark-nlp
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.5/537.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install numpy

In [ ]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()

In [ ]:
# Create a dataframe from the sample_text
data = spark.createDataFrame([
["""As she traveled across the world, Emma visited many different places
and met many fascinating people. She walked the busy streets of Tokyo,
hiked the rugged mountains of Nepal, and swam in the crystal-clear waters
of the Caribbean. Along the way, she befriended locals like Akira, Rajesh,
and Maria, each with their own unique stories to tell. Emma's travels took her
to many cities, including New York, Paris, and Hong Kong, where she savored
delicious foods and explored vibrant cultures. No matter where she went,
Emma always found new wonders to discover and memories to cherish."""]
]).toDF("text")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# PERSON
person_matches = """
Emma
Akira
Rajesh
Maria
"""

with open('person_matches.txt', 'w') as f:
    f.write(person_matches)

# LOCATION
location_matches = """
Tokyo
Nepal
Caribbean
New York
Paris
Hong Kong
"""

with open('location_matches.txt', 'w') as f:
    f.write(location_matches)

In [ ]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline, ReadAs
from sparknlp.annotator import (
    Tokenizer,
    TextMatcher
)
from pyspark.sql.types import StringType
import pyspark.sql.functions as F

In [ ]:
# Step 1: Transforms raw texts to `document` annotation
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

NameError: ignored

In [ ]:
# Step 2: Gets the tokens of the text
tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

In [ ]:
# Step 3: PERSON matcher
person_extractor = TextMatcher() \
    .setInputCols("document", "token") \
    .setEntities("person_matches.txt", ReadAs.TEXT) \
    .setEntityValue("PERSON") \
    .setOutputCol("person_entity") \
    .setCaseSensitive(False)

In [ ]:
# Step 4: LOCATION matcher
location_extractor = TextMatcher() \
    .setInputCols("document", "token") \
    .setEntities("location_matches.txt", ReadAs.TEXT) \
    .setEntityValue("LOCATION") \
    .setOutputCol("location_entity") \
    .setCaseSensitive(False)

In [ ]:
pipeline = Pipeline().setStages([document_assembler,
                                 tokenizer,
                                 person_extractor,
                                 location_extractor
                                 ])

In [ ]:
# Fit and transform to get a prediction
results = pipeline.fit(data).transform(data)

# Display the results
results.selectExpr("person_entity.result").show(truncate=False)

+----------------------------------+
|result                            |
+----------------------------------+
|[Emma, Akira, Rajesh, Maria, Emma]|
+----------------------------------+



In [ ]:
results.selectExpr("location_entity.result").show(truncate=False)

+-----------------------------------------------------+
|result                                               |
+-----------------------------------------------------+
|[Tokyo, Nepal, Caribbean, New York, Paris, Hong Kong]|
+-----------------------------------------------------+



In this example, we created two TextMatcher stages, one matches person names and the other stage matches locations. Once the Spark NLP pipeline is applied to the sample text, any words that match the specified words are extracted.
